# Strategy Performance

This notebook calculates the realzied performance from various implemented strategy

In [219]:
from datetime import datetime
from utils.data import *
from utils.data_helper import *
from pandas.tseries.offsets import BDay
from account.Futu import Futu
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly
from utils.logging import get_logger

pd.options.display.max_rows = 200
pd.options.display.max_columns = 50

logger = get_logger('Portfolio Performance')

In [220]:
start_date = datetime(2023,11,6)
end_date = get_today() + BDay(1)
capital = 20000

# if empty assume all traded strategy
strategy = ''

In [221]:
acc = Futu()
#order_strategy = acc.get_orders_by_strategy(strategy=strategy)

order_hist = acc.order_history(start_date, end_date + BDay(1))
order_hist = order_hist[~order_hist['code'].isin(acc.position_lock)]
order_hist = order_hist.sort_values('create_time')
#order_hist = order_hist[order_hist['order_id'].isin(order_strategy['order_id'].unique())]
#order_hist

2023-12-28 22:09:25,135 | 19073 | [open_context_base.py] _send_init_connect_sync:311: InitConnect ok: conn_id=32, host=127.0.0.1, port=11111, user_id=18214795


2023-12-28 22:09:25,745 | 19073 | [open_context_base.py] on_disconnect:383: Disconnected: conn_id=32


In [222]:
symbol_pnl = {}
df_pnl_close = []

for _, row in order_hist.iterrows():
    code = row['code']
    symbol = code.replace('US.', '')    

    if symbol not in symbol_pnl:
        symbol_pnl[symbol] = {
            'symbol': symbol,
            'qty': 0,
            'pnl': 0,
            'side': 'Long' if 'BUY' in row['trd_side'] else 'Short',
            'open_time': datetime.strptime(row['updated_time'],'%Y-%m-%d %H:%M:%S.%f').date(),    
            'commission': 0,
        }

    qty = row['qty'] if 'BUY' in row['trd_side'] else -row['qty']
    symbol_pnl[symbol]['qty'] += qty    
    symbol_pnl[symbol]['pnl'] += -qty * row['dealt_avg_price']
    symbol_pnl[symbol]['commission'] += -2
    symbol_pnl[symbol]['last_traded_date'] = datetime.strptime(row['updated_time'],'%Y-%m-%d %H:%M:%S.%f').date()
    
    if symbol_pnl[symbol]['qty'] == 0:
        symbol_pnl[symbol]['close_date'] = datetime.strptime(row['updated_time'],'%Y-%m-%d %H:%M:%S.%f').date()
        df_pnl_close.append(symbol_pnl[symbol])
        del symbol_pnl[symbol]
        
df_pnl_close = pd.DataFrame(df_pnl_close)
df_pnl_open = pd.DataFrame(symbol_pnl.values())
df_pnl = pd.concat([df_pnl_close, df_pnl_open])

df_pnl['no_long'] = 1 * (df_pnl['side'] == 'Long')
df_pnl['no_short'] = 1 * (df_pnl['side'] == 'Short')
df_pnl['count'] = 1
df_pnl['winnnig_trades'] = (df_pnl['qty'] == 0) &  (df_pnl['pnl'] > 0)
df_pnl['losing_trades'] = (df_pnl['qty'] == 0) &  (df_pnl['pnl'] < 0)

df_pnl_closed = df_pnl[df_pnl['qty'] == 0]
df_pnl_closed = df_pnl_closed.sort_values('close_date', ascending=False)

logger.info('Total PnL: ${:.0f}'.format(df_pnl_closed['pnl'].sum()))
logger.info('Total Comms: ${:.0f}'.format(df_pnl_closed['commission'].sum()))
logger.info('Number of positive trades: {}'.format(len(df_pnl_closed[df_pnl_closed['pnl'] > 0])))
logger.info('Number of negative trades: {}'.format(len(df_pnl_closed[df_pnl_closed['pnl'] < 0])))

display(df_pnl_closed)

2023-12-28 22:09:25,802 - Portfolio Performance - INFO - Total PnL: $-1985
2023-12-28 22:09:25,805 - Portfolio Performance - INFO - Total Comms: $-634


2023-12-28 22:09:25,812 - Portfolio Performance - INFO - Number of positive trades: 70
2023-12-28 22:09:25,816 - Portfolio Performance - INFO - Number of negative trades: 87


,symbol,qty,pnl,side,open_time,commission,last_traded_date,close_date,no_long,no_short,count,winnnig_trades,losing_trades
156,CIB,0.0,52.6281,Long,2023-12-27,-4,2023-12-27,2023-12-27,1,0,1,True,False
155,KT,0.0,12.9332,Long,2023-12-27,-4,2023-12-27,2023-12-27,1,0,1,True,False
154,NTES,0.0,-42.9885,Long,2023-12-27,-4,2023-12-27,2023-12-27,1,0,1,False,True
153,OXLCL,0.0,-112.2760,Long,2023-12-27,-4,2023-12-27,2023-12-27,1,0,1,False,True
152,XEL,0.0,-18.8110,Long,2023-12-27,-4,2023-12-27,2023-12-27,1,0,1,False,True
151,AZN,0.0,-2.2951,Short,2023-12-27,-4,2023-12-27,2023-12-27,0,1,1,False,True
149,YY,0.0,17.9477,Long,2023-12-22,-4,2023-12-22,2023-12-22,1,0,1,True,False
147,TTE,0.0,15.7122,Short,2023-12-22,-4,2023-12-22,2023-12-22,0,1,1,True,False
148,ANSS,0.0,-298.4234,Short,2023-12-22,-4,2023-12-22,2023-12-22,0,1,1,False,True
150,COLM,0.0,-25.8000,Long,2023-12-22,-4,2023-12-22,2023-12-22,1,0,1,False,True


In [223]:
daily_pnl = df_pnl_closed.groupby(['close_date']).sum(numeric_only=True).reset_index()
daily_pnl['pnl'] = daily_pnl['pnl'] + daily_pnl['commission']
daily_pnl['cum_pnl'] = daily_pnl['pnl'].cumsum()

colors = plotly.colors.DEFAULT_PLOTLY_COLORS

title = f'Portfolio PnL - {strategy}' if len(strategy) else 'Portfolio PnL'
fig = make_subplots(
    5, 2,
    horizontal_spacing = 0.10, 
    vertical_spacing=0.10,
    subplot_titles=[
        'Cumulative PnL', 
        'Cumulative Return %', 
        'Daily PnL',
        'Daily Return %',
        '# Trades',
        'Winning and Losing Trades',
    ],    
    shared_xaxes='all',    
)
fig.update_layout(height=1500, width=1000, title=title)
fig.update_layout(
    xaxis_showticklabels=True, 
    xaxis2_showticklabels=True, 
    xaxis3_showticklabels=True,
    xaxis4_showticklabels=True,
    xaxis5_showticklabels=True,
    legend_tracegroupgap=270,
    hovermode='x',
)

row = 1
fig.add_trace(go.Scatter(x=daily_pnl['close_date'], y=daily_pnl['cum_pnl'], showlegend=True, name='cum pnl', legendgroup=row, marker=dict(color=colors[0])), row=row, col=1)
fig['layout']['yaxis']['title']='USD Dollar $'

fig.add_trace(go.Scatter(x=daily_pnl['close_date'], y=100 * daily_pnl['cum_pnl'] / capital, showlegend=False, name='cum pnl', legendgroup=row, marker=dict(color=colors[0])), row=row, col=2)
fig['layout']['yaxis'+str(row)]['title']='Cum Return %'

row = 2
fig.add_trace(go.Scatter(x=daily_pnl['close_date'], y=daily_pnl['pnl'], showlegend=True, name='daily pnl', legendgroup=row, marker=dict(color=colors[0])), row=row, col=1)
fig['layout']['yaxis'+str(row)]['title']='USD Dollar $'

fig.add_trace(go.Scatter(x=daily_pnl['close_date'], y=100 * daily_pnl['pnl'] / capital, showlegend=False, name='daily pnl', legendgroup=row, marker=dict(color=colors[0])), row=row, col=2)
fig['layout']['yaxis'+str(row)]['title']='Daily Return %'

row = 3
fig.add_trace(go.Scatter(x=daily_pnl['close_date'], y=daily_pnl['count'], showlegend=True, name='#Total', legendgroup=row, marker=dict(color=colors[0])), row=row, col=1)
fig.add_trace(go.Scatter(x=daily_pnl['close_date'], y=daily_pnl['no_long'], showlegend=True, name='#Long', legendgroup=row, marker=dict(color=colors[1])), row=row, col=1)
fig.add_trace(go.Scatter(x=daily_pnl['close_date'], y=daily_pnl['no_short'], showlegend=True, name='#Short', legendgroup=row, marker=dict(color=colors[2])), row=row, col=1)
fig['layout']['yaxis'+str(row)]['title']='Count'

fig.add_trace(go.Scatter(x=daily_pnl['close_date'], y=daily_pnl['winnnig_trades'], showlegend=True, name='#Winning trades', legendgroup=row, marker=dict(color=colors[3])), row=row, col=2)
fig.add_trace(go.Scatter(x=daily_pnl['close_date'], y=daily_pnl['losing_trades'], showlegend=True, name='#Losing trades', legendgroup=row, marker=dict(color=colors[4])), row=row, col=2)
fig['layout']['yaxis'+str(row)]['title']='Count'

fig.show()